In [0]:
# a = []
# while(1):
#     a.append('1')

In [0]:
!pip install -U -q kaggle
!mkdir ~/.kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nikunjgoel","key":"4d4b8ceb591a64c1ae4e80996446b150"}'}

In [0]:
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c 11-785-s20-hw2p2-classification

  0% 0.00/53.9k [00:00<?, ?B/s]
100% 53.9k/53.9k [00:00<00:00, 48.5MB/s]
 98% 577M/589M [00:06<00:00, 76.0MB/s]
100% 589M/589M [00:06<00:00, 99.0MB/s]


In [0]:
!unzip 11-785-hw2p2-s20.tgz.zip
!tar zxvf 11-785-hw2p2-s20.tgz
# !unzip /content/11-785hw2p2-s20/train_data.zip

Archive:  11-785-hw2p2-s20.tgz.zip
  inflating: 11-785-hw2p2-s20.tgz    
._11-785hw2p2-s20
11-785hw2p2-s20/
11-785hw2p2-s20/._validation_verification.zip
11-785hw2p2-s20/validation_verification.zip
11-785hw2p2-s20/._test_classification.zip
11-785hw2p2-s20/test_classification.zip
11-785hw2p2-s20/._validation_classification.zip
11-785hw2p2-s20/validation_classification.zip
11-785hw2p2-s20/._validation_trials_verification.txt
11-785hw2p2-s20/validation_trials_verification.txt
11-785hw2p2-s20/test_trials_verification_student.txt
11-785hw2p2-s20/._test_order_classification.txt
11-785hw2p2-s20/test_order_classification.txt
11-785hw2p2-s20/._test_verification.zip
11-785hw2p2-s20/test_verification.zip
11-785hw2p2-s20/._train_data.zip
11-785hw2p2-s20/train_data.zip


In [0]:
!unzip /content/11-785hw2p2-s20/validation_verification.zip
!unzip /content/11-785hw2p2-s20/test_verification.zip

In [0]:
import torch
import torch.nn.functional as F
from torch import autograd, nn
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, datasets

In [0]:
# Use torch ImageFolder
data_transform = transforms.Compose([
        transforms.ToTensor()
    ])
face_dataset = datasets.ImageFolder(root='/content/train_data/medium/',
                                           transform=data_transform)
dataset_loader = torch.utils.data.DataLoader(face_dataset,
                                             batch_size=512,
                                             shuffle=True,
                                             num_workers=8)

In [0]:
class BasicCNNModule(nn.Module):
    def __init__(self):
        super(BasicCNNModule, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 32 * 24 * 24)
        return x
        
print(BasicCNNModule())

BasicCNNModule(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
)


In [0]:
model = BasicCNNModule()
cuda = torch.cuda.is_available()
cuda
device = torch.device("cuda" if cuda else "cpu")
device

device(type='cuda')

In [0]:
import torch.optim as optim
import time
import pandas as pd
import os
from PIL import Image
import sys

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode = 'min',factor=0.1,patience=2)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)

BasicCNNModule(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
)


In [0]:
checkpoint = torch.load('/content/drive/My Drive/trained_model.pth')

In [0]:
model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

RuntimeError: ignored

In [0]:
model.state_dict()

In [0]:
data_transform = transforms.Compose([
        #transforms.RandomResizedCrop(100),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])

class TestingDataset(torch.utils.data.Dataset):

    def __init__(self,text_file,root_dir,transform=transforms.ToTensor()):
       
        self.name_frame = pd.read_csv(text_file,sep=" ",header=None)
        #self.label_frame = pd.read_csv(text_file,sep=" ")
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.name_frame)

    def __getitem__(self, idx):
        img_name1, img_name2 = self.name_frame.iloc[idx]
        d = os.path.join(self.root_dir,img_name1)
        image1 = Image.open(d)
        image1 = self.transform(image1)
        d = os.path.join(self.root_dir,img_name2)
        image2 = Image.open(d)
        image2 = self.transform(image2)
        #labels = self.label_frame.iloc[idx, 0]
        #labels = labels.reshape(-1, 2)
        # sample = {'image': image, 'labels': labels}
        #sample = {image1, image2}

        return img_name1, img_name2, image1, image2


In [0]:
test_verification = TestingDataset('/content/11-785hw2p2-s20/test_trials_verification_student.txt','/content/test_verification/')
test_dataset_loader = torch.utils.data.DataLoader(test_verification,
                                             batch_size=1024,
                                             shuffle=False,
                                             num_workers=8)

In [0]:
def test_model(model, test_loader):
    with torch.no_grad():
        model.eval()

        # running_loss = 0.0
        # total_predictions = 0.0
        # correct_predictions = 0.0
        # output_l = []
        # target_l = []
        cos_l = []

        for batch_idx, (img1, img2,data, target) in enumerate(test_loader):   
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)

            _, predicted1 = torch.max(outputs.data, 1)

            outputs = model(target)

            _, predicted2 = torch.max(outputs.data, 1)

            for i in range(len(img1)):
              pred1cpu = predicted1[i].data.cpu()
              pred2cpu = predicted2[i].data.cpu()

            #cosine_value = F.cosine_similarity(predicted1, predicted2)
              cos = F.cosine_similarity(pred1cpu.float(), pred2cpu.float(), dim=0, eps=1e-8)
            #cosine_value = cos(predicted1.data.cpu(), predicted2.data.cpu())
            #print(cos)
              # if cos > 0.75:
              #   cos = 1
              # else:
              #   cos = 0
              cos_l.append((img1[i]+' '+img2[i],cos))
            
            # o = predicted.data.cpu().numpy()

            # output_l.append(np.array(o))
            # target_l.append(np.array(target))
            # total_predictions += target.size(0)
            # correct_predictions += (predicted == target).sum().item()

            # loss = criterion(outputs, target).detach()
            # running_loss += loss.item()


        # running_loss /= len(test_loader)
        # acc = (correct_predictions/total_predictions)*100.0
        # print('Testing Loss: ', running_loss)
        # print('Testing Accuracy: ', acc, '%')
        # return running_loss, acc
        return np.array(cos_l)

In [0]:
cosine_list = test_model(model, test_dataset_loader)

In [0]:
FinalOutput = pd.DataFrame(cosine_list, columns=['trial','score'])

In [0]:
FinalOutput.to_csv('/content/FinalOutput.csv',index=False)

In [0]:
!kaggle competitions submit -c 11-785-s20-hw2p2-verification -f FinalOutput.csv -m "First Submission threshold 0.80"

100% 20.6M/20.6M [00:08<00:00, 2.54MB/s]
Successfully submitted to 11-785, Spring 2020, HW2P2 - Verification